# Flash Point: Fire Rescue

## Instalación e importación de librerías

In [7]:
# Descargar e instalar mesa y seaborn

%pip install mesa==2.3.1 seaborn --quiet

Note: you may need to restart the kernel to use updated packages.


In [8]:
# Importamos las clases que se requieren para manejar los agentes (Agent) y su entorno (Model).
# Cada modelo puede contener múltiples agentes.
from mesa import Agent, Model

# Debido a que necesitamos que existe un solo agente por celda, elegimos ''SingleGrid''.
from mesa.space import SingleGrid

# Con 'SimultaneousActivation' podemos activar todos los agentes al mismo tiempo.
from mesa.time import SimultaneousActivation

# Con 'RandomActivation' podemos activar los agentes en un orden aleatorio.
from mesa.time import RandomActivation

# Haremos uso de ''DataCollector'' para obtener información de cada paso de la simulación.
from mesa.datacollection import DataCollector

# BATCH_RUNNER
from mesa.batchrunner import batch_run

# matplotlib lo usaremos crear una animación de cada uno de los pasos del modelo.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Importamos los siguientes paquetes para el mejor manejo de valores numéricos.
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime

# Importamos el paquete seaborn para mejorar la visualización de los datos.
import seaborn as sns

## Leer archivo de configuración inicial

In [9]:
# Abrir el archivo de configuración
file = open("board-config.txt", "r")

# Obtener la información del tablero
config = file.readlines()

# Cerrar el archivo
file.close()

# Crear un diccionario para almacenar la configuración del tablero
board_config = {}

# Configuración del tablero
board_config['board'] = [x.replace("\n", "").split() for x in config[:6]]

# Puntos de interés
board_config['points_of_interest'] = [x.replace("\n", "").split() for x in config[6:9]]

# Indicadores de fuego
board_config['fire_indicators'] = [x.replace("\n", "").split() for x in config[9:19]]

# Puertas
board_config['doors'] = [x.replace("\n", "").split() for x in config[19:27]]

# Puntos de entrada
board_config['entry_points'] = [x.replace("\n", "").split() for x in config[27:31]]

# Imprimir la configuración del tablero
board_config


{'board': [['1100', '1000', '1001', '1100', '1001', '1100', '1000', '1001'],
  ['0100', '0000', '0011', '0100', '0011', '0110', '0010', '0011'],
  ['0100', '0001', '1100', '1000', '1000', '1001', '1100', '1001'],
  ['0100', '0011', '0110', '0010', '0010', '0011', '0110', '0011'],
  ['1100', '1000', '1000', '1000', '1001', '1100', '1001', '1101'],
  ['0110', '0010', '0010', '0010', '0011', '0110', '0011', '0111']],
 'points_of_interest': [['2', '4', 'v'], ['5', '1', 'f'], ['5', '8', 'v']],
 'fire_indicators': [['2', '2'],
  ['2', '3'],
  ['3', '2'],
  ['3', '3'],
  ['3', '4'],
  ['3', '5'],
  ['4', '4'],
  ['5', '6'],
  ['5', '7'],
  ['6', '6']],
 'doors': [['1', '3', '1', '4'],
  ['2', '5', '2', '6'],
  ['2', '8', '3', '8'],
  ['3', '2', '3', '3'],
  ['4', '4', '5', '4'],
  ['4', '6', '4', '7'],
  ['6', '5', '6', '6'],
  ['6', '7', '6', '8']],
 'entry_points': [['1', '6'], ['3', '1'], ['4', '8'], ['6', '3']]}